# Understanding Portfolio Files

In [8]:
import pandas as pd

## Buda's "Cartola"

Buda is my starting point with crypto investments, I will transform the Excel file into
something I'm more comfortable with (.csv/Dataframe).

In [9]:
fname = 'buda'
buda = pd.read_excel(rf'C:\Users\Kristofher\PycharmProjects\ugly-terminal\portfolio\{fname}.xlsx')
buda.to_csv(f'{fname}.csv', header=True)
buda.head()

,id,operacion,fecha,moneda,monto,balance,detalle,mercado,precio
0,1891b6eb-d873-41de-bce2-07424be173fc,09a65493-a7a4-4a02-92a2-9ac434036653,2021-03-11 14:53:58 UTC,CLP,-700000.000000,172772.700000,compra,ETH-CLP,1310000.0
1,7f568561-3caf-4f4d-89d2-66c8bc07436e,09a65493-a7a4-4a02-92a2-9ac434036653,2021-03-11 14:53:58 UTC,ETH,-0.001870,0.972892,comision compra,ETH-CLP,NaN
2,b8558503-edbd-4b02-aae0-209ade360b9a,09a65493-a7a4-4a02-92a2-9ac434036653,2021-03-11 14:53:58 UTC,ETH,0.534351,0.974762,compra,ETH-CLP,1310000.0
3,109a087a-29ba-4fa8-9cce-8f663def9f7c,105ca170-733f-4a39-b578-e1310b1c41a2,2021-03-11 14:43:15 UTC,ETH,-0.011440,0.440411,comision retiro,NaN,NaN
4,89dc908d-972e-41b9-875d-d5c3739f4be6,105ca170-733f-4a39-b578-e1310b1c41a2,2021-03-11 14:43:15 UTC,ETH,-0.500000,0.451851,retiro,NaN,NaN


In [10]:
# Create a buda-lite csv file
buda_lite = buda.copy()
buda_lite.drop(['id', 'balance', 'mercado'], inplace=True, axis=1)
buda_lite.head()

,operacion,fecha,moneda,monto,detalle,precio
0,09a65493-a7a4-4a02-92a2-9ac434036653,2021-03-11 14:53:58 UTC,CLP,-700000.000000,compra,1310000.0
1,09a65493-a7a4-4a02-92a2-9ac434036653,2021-03-11 14:53:58 UTC,ETH,-0.001870,comision compra,NaN
2,09a65493-a7a4-4a02-92a2-9ac434036653,2021-03-11 14:53:58 UTC,ETH,0.534351,compra,1310000.0
3,105ca170-733f-4a39-b578-e1310b1c41a2,2021-03-11 14:43:15 UTC,ETH,-0.011440,comision retiro,NaN
4,105ca170-733f-4a39-b578-e1310b1c41a2,2021-03-11 14:43:15 UTC,ETH,-0.500000,retiro,NaN


# Simplify Buda file
The Buda file uses three lines to generate a transaction, that's wasteful and confusing,
I will adapt to use delta file.

The Delta .csv file has the following structure:

| Date                       | Type     | Exchange | Base amount | Base currency | Quote amount | Quote currency | Fee  | Fee currency | Costs/Proceeds | Costs/Proceeds currency | Sync Holdings | Sent/Received from | Sent to | Notes                                              |
|----------------------------|----------|----------|-------------|---------------|--------------|----------------|------|--------------|----------------|-------------------------|---------------|--------------------|---------|----------------------------------------------------|
| 2018-01-01 20:40:00 +01:00 | DEPOSIT  | Coinbase | 10000.00    | EUR           | Nan          | Nan            | Nan  | Nan          | Nan            | Nan                     | 0             | Nan                | Nan     | Example of fiat deposit.                           |
| 2018-01-08 17:58:00 +01:00 | BUY      | Coinbase | 2.00        | BTC           | 10000.00     | EUR            | 0.10 | BTC          | Nan            | Nan                     | 1             | Nan                | Nan     | 1.9 BTC will be added. 10000 EUR will be deducted. |
| 2018-01-10 18:00:00 +01:00 | TRANSFER | Nan      | 1.90        | BTC           | Nan          | Nan            | 0.01 | BTC          | Nan            | Nan                     | 0             | Coinbase           | Binance | Example of TRANSFER. Only 0.01 will be deducted.   |
| 2018-01-25 15:05:00 +01:00 | BUY      | Binance  | 20.00       | ETH           | 1.00         | BTC            | 0.02 | BTC          | Nan            | Nan                     | 1             | Nan                | Nan     | 20 ETH will be added and 1.02 BTC deducted.        |
| 2018-02-28 23:59:00 +01:00 | SELL     | Binance  | 10.00       | ETH           | 1.05         | BTC            | 0.50 | ETH          | Nan            | Nan                     | 1             | Nan                | Nan     | 10.5 ETH will be deducted and 1.05 BTC added.      |
| 2018-03-28 22:22:00 +02:00 | WITHDRAW | Nan      | 8.50        | ETH           | Nan          | Nan            | Nan  | Nan          | Nan            | Nan                     | Nan           | Binance            | OTHER   | Example of WITHDRAW. 8.5 ETH will be deducted.     |


In [11]:
index = 0
container = []
while index < len(buda_lite):

    if buda_lite['detalle'].iloc[index] == 'compra':  # we enter the BUY phase

        if buda_lite['detalle'].iloc[index + 1] == 'comision compra':  # there's a fee
            has_fee = 1
        else:
            has_fee = 0

        if buda_lite['monto'].iloc[index] >= 0:
            pos_index = index
            neg_index = index + 1 + has_fee
        else:
            pos_index = index + 1 + has_fee
            neg_index = index

        # fill values
        date = buda_lite['fecha'].iloc[index]
        typ = 'BUY'
        exchange = 'Buda'
        base_amount = buda_lite['monto'].iloc[pos_index]
        base_currency = buda_lite['moneda'].iloc[pos_index]
        quote_amount = abs(buda_lite['monto'].iloc[neg_index])
        quote_currency = buda_lite['moneda'].iloc[neg_index]

        if has_fee:
            fee = abs(buda_lite['monto'].iloc[index + 1])
            fee_currency = buda_lite['moneda'].iloc[index + 1]
        else:
            fee = None
            fee_currency = None

        cost_proceeds = None
        cost_proceeds_currency = None
        sync = 1
        sent_received_from = None
        sent_to = None
        notes = buda_lite['operacion'].iloc[index]
        # and advance
        index = index + 2 + has_fee

    elif buda_lite['detalle'].iloc[index] == 'abono':  # enter DEPOSIT phase
        date = buda_lite['fecha'].iloc[index]
        typ = 'DEPOSIT'
        exchange = 'Buda'
        base_amount = buda_lite['monto'].iloc[index]
        base_currency = buda_lite['moneda'].iloc[index]
        quote_amount = None
        quote_currency = None
        fee = None
        fee_currency = None
        cost_proceeds = None
        cost_proceeds_currency = None
        sync = 0
        sent_received_from = 'OTHER'
        sent_to = 'Buda'
        notes = buda_lite['operacion'].iloc[index]
        # advance
        index = index + 1

    elif buda_lite['detalle'].iloc[index] == 'comision venta':  # enter SELL phase
        pos_index = None
        if buda_lite['monto'].iloc[index + 1] >= 0:
            pos_index = index + 1
            neg_index = index + 2
        else:
            pos_index = index + 2
            neg_index = index + 1

        # fill values
        date = buda_lite['fecha'].iloc[index]
        typ = 'SELL'
        exchange = 'Buda'
        base_amount = abs(buda_lite['monto'].iloc[neg_index])
        base_currency = buda_lite['moneda'].iloc[neg_index]
        quote_amount = buda_lite['monto'].iloc[pos_index]
        quote_currency = buda_lite['moneda'].iloc[pos_index]
        fee = abs(buda_lite['monto'].iloc[index])
        fee_currency = buda_lite['moneda'].iloc[index]
        cost_proceeds = None
        cost_proceeds_currency = None
        sync = 1
        sent_received_from = None
        sent_to = None
        notes = buda_lite['operacion'].iloc[index]
        # and advance
        index = index + 3

    # TODO: In case of a transfer or withdraw phase, for now we use to MY_WALLET as placeholder
    # to fill where that money went. Having other files will let us know where that money went.
    elif buda_lite['detalle'].iloc[index] == 'comision retiro':  # enter TRANSFER phase
        date = buda_lite['fecha'].iloc[index]
        typ = 'TRANSFER'
        exchange = None
        base_amount = abs(buda_lite['monto'].iloc[index + 1])
        base_currency = buda_lite['moneda'].iloc[index + 1]
        quote_amount = None
        quote_currency = None
        fee = abs(buda_lite['monto'].iloc[index])
        fee_currency = buda_lite['moneda'].iloc[index]
        cost_proceeds = None
        cost_proceeds_currency = None
        sync = 0
        sent_received_from = 'Buda'
        sent_to = 'MY_WALLET'  # we need to update this value with other info: etherscan and binance
        notes = buda_lite['operacion'].iloc[index]
        # advance
        index = index + 2

    elif buda_lite['detalle'].iloc[index] == 'retiro':  # enter WITHDRAW phase
        date = buda_lite['fecha'].iloc[index]
        typ = 'WITHDRAW'
        exchange = 'Buda'
        base_amount = abs(buda_lite['monto'].iloc[index])
        base_currency = buda_lite['moneda'].iloc[index]
        quote_amount = None
        quote_currency = None
        fee = None
        fee_currency = None
        cost_proceeds = None
        cost_proceeds_currency = None
        sync = 1
        sent_received_from = 'Buda'
        sent_to = 'OTHER'
        notes = buda_lite['operacion'].iloc[index]
        # advance
        index = index + 1

    else:  # just keep advancing
        index = index + 1
        continue

    # append values, each statement must fill those values
    container.append([date, typ, exchange, base_amount, base_currency, quote_amount,
                      quote_currency, fee, fee_currency, cost_proceeds, cost_proceeds_currency,
                      sync, sent_received_from, sent_to, notes])


df = pd.DataFrame(container, columns=['Date', 'Type', 'Exchange', 'Base amount', 'Base currency',
                                      'Quote amount', 'Quote currency', 'Fee', 'Fee currency',
                                      'Costs/Proceeds', 'Costs/Proceeds currency', 'Sync holdings',
                                      'Sent/Received from', 'Sent to', 'Notes'])
df.head(30)

,Date,Type,Exchange,Base amount,Base currency,Quote amount,Quote currency,Fee,Fee currency,Costs/Proceeds,Costs/Proceeds currency,Sync holdings,Sent/Received from,Sent to,Notes
0,2021-03-11 14:53:58 UTC,BUY,Buda,5.343511e-01,ETH,700000.00,CLP,1.870229e-03,ETH,None,None,1,None,None,09a65493-a7a4-4a02-92a2-9ac434036653
1,2021-03-11 14:43:15 UTC,TRANSFER,None,5.000000e-01,ETH,NaN,None,1.144000e-02,ETH,None,None,0,Buda,MY_WALLET,105ca170-733f-4a39-b578-e1310b1c41a2
2,2021-03-11 14:28:52 UTC,DEPOSIT,Buda,7.000000e+05,CLP,NaN,None,NaN,None,None,None,0,OTHER,Buda,47574cff-74f2-493b-83b7-fd46890cbcfc
3,2021-03-11 13:56:12 UTC,TRANSFER,None,5.300000e-01,ETH,NaN,None,1.000000e-02,ETH,None,None,0,Buda,MY_WALLET,579833f9-e43f-42b5-ac85-f24fd1c44813
4,2021-03-11 13:23:41 UTC,BUY,Buda,5.323194e-01,ETH,700000.00,CLP,1.863117e-03,ETH,None,None,1,None,None,2d1e9e70-aa8f-48a6-8b70-055ea3f574e5
5,2021-03-11 12:58:36 UTC,DEPOSIT,Buda,7.000000e+05,CLP,NaN,None,NaN,None,None,None,0,OTHER,Buda,e1ce6ccd-bbf0-47ac-bc3b-8ebb7f3e78f5
6,2021-02-28 05:26:29 UTC,BUY,Buda,6.378940e-02,ETH,64491.08,CLP,2.232620e-04,ETH,None,None,1,None,None,091c4ff3-97e9-4fa0-896f-e4797c6849d7
7,2021-02-28 05:14:15 UTC,BUY,Buda,9.143550e-03,BTC,299999.88,CLP,3.200000e-05,BTC,None,None,1,None,None,3c36f013-dd9b-44b4-a645-0c54ea9827ca
8,2021-02-28 05:05:54 UTC,BUY,Buda,1.340345e-01,ETH,135508.92,CLP,9.382410e-04,ETH,None,None,1,None,None,f06565db-5841-4e42-9502-ccd2f7046b7b
9,2021-02-28 04:58:57 UTC,DEPOSIT,Buda,5.000000e+05,CLP,NaN,None,NaN,None,None,None,0,OTHER,Buda,e47b5b03-ccc4-4c36-8510-4b3378ca98f0


## Exporting Dataframe to a Delta csv

In [12]:
name_out = f'{fname}_proc_buda.csv'
print(f'Saving file as {name_out}')
df.to_csv(name_out, index=False)

Saving file as buda_proc_buda.csv


## Adding Binance Trades
Let's load them up!

In [13]:
binance_in = 'binance_trades.csv'
df_binance = pd.read_csv(binance_in)

df_binance.head()

,Date(UTC),Pair,Side,Price,Executed,Amount,Fee
0,2021-03-31 15:15:41,RUNEUSDT,SELL,7.90000,79.7500000000RUNE,630.02500000USDT,0.6300250000USDT
1,2021-03-18 10:39:45,LINKUSDT,SELL,30.01030,17.2400000000LINK,517.37757200USDT,0.5173775700USDT
2,2021-03-16 12:35:51,GRTUSDT,BUY,1.67499,351.5000000000GRT,588.75898500USDT,0.3515000000GRT
3,2021-03-12 10:29:41,LINKUSDT,BUY,28.77120,17.5200000000LINK,504.07142400USDT,0.0175200000LINK
4,2021-03-11 21:16:40,LINKUSDT,BUY,29.78000,18.8300000000LINK,560.75740000USDT,0.0188300000LINK


In [14]:
# Define trade/groups: USDT, USDC, DAI, BTC, ETH as currency use to buy stuff
# In this way we can separate the trade pairs into two
# In case of BTC-USDT, USDT is taken as the base as it's first in priority in bases array
bases = ['USDT', 'USDC', 'DAI', 'BTC', 'ETH']

for index, pair in enumerate(df_binance['Pair']):

    # Separate pair into base and coin
    my_base = None
    my_coin = None

    for base in bases:
        if base in pair:
            my_base = base
            my_coin = pair.replace(base, '')
            break

    # now we use base/coin strings to parse amounts taking out the string part and erasing commas
    executed = float(df_binance['Executed'].iloc[index].replace(my_coin, '').replace(',', ''))
    amount = float(df_binance['Amount'].iloc[index].replace(my_base, '').replace(',', ''))
    fee = float(df_binance['Fee'].iloc[index].replace(my_coin, '').replace(my_base, '').replace(',', ''))

    # fill values
    date = df_binance['Date(UTC)'].iloc[index]
    typ = df_binance['Side'].iloc[index]
    exchange = 'Binance'
    base_amount = executed
    base_currency = my_coin
    quote_amount = amount
    quote_currency = my_base
    if typ == 'BUY':
        fee_currency = my_coin
    elif typ == 'SELL':
        fee_currency = my_base
    else:
        fee_currency = None
    cost_proceeds = None
    cost_proceeds_currency = None
    sync = 1
    sent_received_from = None
    sent_to = None
    notes = 'Binance transaction'

    # append values
    container.append([date, typ, exchange, base_amount, base_currency, quote_amount,
                      quote_currency, fee, fee_currency, cost_proceeds, cost_proceeds_currency,
                      sync, sent_received_from, sent_to, notes])

df = pd.DataFrame(container, columns=['Date', 'Type', 'Exchange', 'Base amount', 'Base currency',
                                      'Quote amount', 'Quote currency', 'Fee', 'Fee currency',
                                      'Costs/Proceeds', 'Costs/Proceeds currency', 'Sync holdings',
                                      'Sent/Received from', 'Sent to', 'Notes'])

name_out = f'{fname}_proc_binance.csv'
print(f'Saving file as {name_out}')
df.to_csv(name_out, index=False)

Saving file as buda_proc_binance.csv


In [15]:
df.head(100)

,Date,Type,Exchange,Base amount,Base currency,Quote amount,Quote currency,Fee,Fee currency,Costs/Proceeds,Costs/Proceeds currency,Sync holdings,Sent/Received from,Sent to,Notes
0,2021-03-11 14:53:58 UTC,BUY,Buda,0.534351,ETH,700000.000000,CLP,0.001870,ETH,None,None,1,None,None,09a65493-a7a4-4a02-92a2-9ac434036653
1,2021-03-11 14:43:15 UTC,TRANSFER,None,0.500000,ETH,NaN,None,0.011440,ETH,None,None,0,Buda,MY_WALLET,105ca170-733f-4a39-b578-e1310b1c41a2
2,2021-03-11 14:28:52 UTC,DEPOSIT,Buda,700000.000000,CLP,NaN,None,NaN,None,None,None,0,OTHER,Buda,47574cff-74f2-493b-83b7-fd46890cbcfc
3,2021-03-11 13:56:12 UTC,TRANSFER,None,0.530000,ETH,NaN,None,0.010000,ETH,None,None,0,Buda,MY_WALLET,579833f9-e43f-42b5-ac85-f24fd1c44813
4,2021-03-11 13:23:41 UTC,BUY,Buda,0.532319,ETH,700000.000000,CLP,0.001863,ETH,None,None,1,None,None,2d1e9e70-aa8f-48a6-8b70-055ea3f574e5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2021-03-11 21:14:49,BUY,Binance,6.060000,LINK,180.466800,USDT,0.006060,LINK,None,None,1,None,None,Binance transaction
96,2021-03-11 12:36:36,SELL,Binance,79.750000,RUNE,528.016775,USDT,0.528017,USDT,None,None,1,None,None,Binance transaction
97,2021-03-10 22:46:50,BUY,Binance,159.660000,RUNE,1026.821358,USDT,0.159660,RUNE,None,None,1,None,None,Binance transaction
98,2021-03-10 16:59:34,SELL,Binance,0.008531,BTC,486.778860,USDT,0.486779,USDT,None,None,1,None,None,Binance transaction


## Calculating total balances

In [31]:
def update_coin(_dict, _coin, _value):
    if not _coin or not _value:  # either no coin or no value
        return

    try:  # if it exist we update value
        _dict[_coin] = _dict[_coin] + _value
    except KeyError:  # if it doesn't we create a new one
        _dict[_coin] = _value

    return

In [32]:
total = dict()
for _, line in df.iterrows():
    # first of all, if not a BUY, SELL or DEPOSIT we continue
    if line['Type'] not in ['BUY', 'SELL', 'DEPOSIT']:
        continue

    # if SELL we invert signs
    if line['Type'] == 'SELL':
        sign = -1
    else:
        sign = 1

    update_coin(total, line['Base currency'], line['Base amount'] * sign)
    update_coin(total, line['Quote currency'], line['Quote amount'] * -sign)
    update_coin(total, line['Fee currency'], -line['Fee'])

print(total)

{'ETH': 6.342328191000003, 'CLP': 175728.37, 'BTC': 0.02911931000000001, 'RUNE': 0.00033999999999659303, 'USDT': 2183.42698241, 'LINK': 54.60569999999999, 'GRT': 351.23920000000004, 'RSR': 0.9962000000021135, 'ATOM': 5.010984, 'USDC': 0.009352359999790005, 'MKR': 0.0007469999999999852, 'AAVE': 0.0059900000000000795, 'CRV': 0.0028799999999590264, 'YFI': 7.83e-05, 'FET': 266.733, 'COTI': 0.99500000000001, '1INCH': 0.009140000000007947, 'TOMO': -0.04000000000000034, 'DOT': 0.07200000000000062, 'OCEAN': 24.878999999999994, 'SXP': 1.998, 'REN': 0.8839999999999999, 'IDEX': 7.97, 'DOGE': 419.58}
